In [1]:
import numpy as np

In [2]:
def get_input_shape(input_matrix_shape, filter_matrix_shape):
    matrix_dim = len(input_matrix_shape)
    filter_dim = len(filter_matrix_shape)
    
    if(matrix_dim == 3):
        input_depth, input_height, input_width = input_matrix_shape
    else:
        input_height, input_width =input_matrix_shape
        input_depth = 1
        
    if(filter_dim == 3):
        filter_depth, filter_height, filter_width = filter_matrix_shape
    else:
        filter_height, filter_width = filter_matrix_shape
        filter_depth = 1 
        
    return input_depth, input_height, input_width, filter_depth, filter_height, filter_width

### Прямая реализация свертки

In [3]:
def direct_conv(input_matrix, filter_matrix):
    input_depth, input_height, input_width, \
        filter_depth, filter_height, filter_width = get_input_shape(input_matrix.shape, filter_matrix.shape)
    
    output_depth = input_depth - filter_depth + 1
    output_height = input_height - filter_height + 1
    output_width = input_width - filter_width + 1
    
    output_matrix = np.zeros((output_depth, output_height, output_width))

    for d in range(output_depth):
        for i in range(output_height):
            for j in range(output_width):
                if(input_depth != 1):
                    frame = input_matrix[d:d+filter_depth, i:i+filter_height, j:j+filter_width]
                else:
                    frame = input_matrix[i:i+filter_height, j:j+filter_width]
                output_matrix[d, i, j] = np.sum(frame * filter_matrix)

    return output_matrix

### Реализация свертки через im2col

In [4]:
def im2col(input_matrix, filter_shape = None):
    if filter_shape == None:
        return input_matrix.flatten()
    
    input_depth, input_height, input_width, \
        filter_depth, filter_height, filter_width = get_input_shape(input_matrix.shape, filter_shape)
        
    output_height = input_height - filter_height + 1
    output_width = input_width - filter_width + 1

    output = np.zeros((filter_height * filter_width * input_depth, output_height * output_width))

    for d in range(input_depth):
        for i in range(output_height):
            for j in range(output_width):
                if(input_depth != 1):
                    frame = input_matrix[d, i:i + filter_height, j:j + filter_width]
                else: 
                    frame = input_matrix[i:i + filter_height, j:j + filter_width]
                output[d * filter_height * filter_width : (d + 1) * filter_height * filter_width,
                           i * output_width + j] = frame.flatten()

    return output.T

In [5]:
def im2col_conv(input_matrix, filter_matrix):
    input_depth, input_height, input_width, \
        filter_depth, filter_height, filter_width = get_input_shape(input_matrix.shape,filter_matrix.shape)
    
    output_height = input_height - filter_height + 1
    output_width = input_width - filter_width + 1
    
    res = np.dot(im2col(input_matrix, filter_matrix.shape), im2col(filter_matrix))
    
    if (len(input_matrix.shape) == 3) :    
        return res.reshape(-1, output_height, output_width)
    else:
        return res.reshape(output_height, output_width)

### Сиа

In [6]:
input_matrix_2d = np.random.randint(0, 10, size=(6, 6), dtype=int)
filter_matrix_2d = np.array([[0, -1, 0],
                             [-1, 5, -1], 
                             [0, -1, 0]])

print("Input matrix:\n", input_matrix_2d)
print()
print("Filter matrix:\n", filter_matrix_2d)

Input matrix:
 [[2 0 0 8 6 8]
 [9 4 3 0 1 0]
 [8 4 4 5 3 1]
 [4 9 4 7 4 2]
 [3 8 2 9 9 7]
 [2 2 4 7 6 9]]

Filter matrix:
 [[ 0 -1  0]
 [-1  5 -1]
 [ 0 -1  0]]


In [7]:
print("Direct convolutional 2d:\n", direct_conv(input_matrix_2d, filter_matrix_2d))

Direct convolutional 2d:
 [[[  4.   7. -17.  -4.]
  [ -5.   4.  11.   4.]
  [ 25.  -2.  13.  -1.]
  [ 24. -15.  20.  19.]]]


In [8]:
print("Im2col convolutional 2d:\n", im2col_conv(input_matrix_2d, filter_matrix_2d))

Im2col convolutional 2d:
 [[  4.   7. -17.  -4.]
 [ -5.   4.  11.   4.]
 [ 25.  -2.  13.  -1.]
 [ 24. -15.  20.  19.]]


In [9]:
input_matrix_3d = np.random.randint(0, 10, size=(2, 5, 5), dtype=int)
filter_matrix_3d = np.array(([[0, -1], [1, 0]],
                             [[0, 1], [0, -1]]))

print("Input matrix:\n", input_matrix_3d)
print()
print("Filter matrix:\n", filter_matrix_3d)

Input matrix:
 [[[2 2 0 1 2]
  [6 3 5 6 5]
  [5 5 0 8 9]
  [3 8 7 0 0]
  [5 5 7 7 2]]

 [[1 8 7 3 6]
  [2 9 1 7 9]
  [8 6 7 3 8]
  [2 7 9 2 4]
  [0 8 6 9 8]]]

Filter matrix:
 [[[ 0 -1]
  [ 1  0]]

 [[ 0  1]
  [ 0 -1]]]


In [10]:
print("Direct convolutional 3d:\n", direct_conv(input_matrix_3d, filter_matrix_3d))

Direct convolutional 3d:
 [[[ 3.  9.  0.  1.]
  [ 5. -6. -2.  4.]
  [-3.  6.  0. -5.]
  [-4.  1.  0.  3.]]]


In [11]:
print("Im2col convolutional 3d:\n", im2col_conv(input_matrix_3d, filter_matrix_3d))

Im2col convolutional 3d:
 [[[ 3.  9.  0.  1.]
  [ 5. -6. -2.  4.]
  [-3.  6.  0. -5.]
  [-4.  1.  0.  3.]]]
